In [6]:
import yfinance as yf
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Define the ticker symbols
tickers = ['XOM', 'CVX']

# Download historical stock price data
data = yf.download(tickers, start='2019-01-01', end='2023-01-01')['Adj Close']

# Calculate normalized return index for each ticker
returns = (data / data.iloc[0])

# Calculate squared deviations between normalized price series
squared_deviations = (returns['XOM'] - returns['CVX']) ** 2

# Calculate the normalized price distance
normalized_price_distance = squared_deviations.sum()

# Display the result
print("Normalized Price Distance:", normalized_price_distance)


# Calculate the price spread (log difference)
data['Price_Spread'] = np.log(data['XOM']) - np.log(data['CVX'])

# Perform linear regression
X = sm.add_constant(np.log(data['CVX']))
model = sm.OLS(data['Price_Spread'], X).fit()

# Get residuals
residuals = model.resid


# Perform ADF test
adf_test = adfuller(residuals)
p_value = adf_test[1]


if p_value < 0.05:
    print("The residuals are stationary. The stocks are likely cointegrated.")
else:
    print("The residuals are not stationary. The stocks are likely not cointegrated.")

[*********************100%***********************]  2 of 2 completed
Normalized Price Distance: 14.863513028153172
The residuals are stationary. The stocks are likely cointegrated.


### Setting open and exit threshold and strategy execution

In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

# Download historical price data for XOM and CVX
start_date = '2019-01-01'
end_date = '2023-01-01'
xom_data = yf.download('XOM', start=start_date, end=end_date)['Adj Close']
cvx_data = yf.download('CVX', start=start_date, end=end_date)['Adj Close']

# Compute the spread between XOM and CVX prices
spread = xom_data - cvx_data

# Calculate mean and standard deviation of the spread
mean_spread = np.mean(spread)
std_spread = np.std(spread)

# Compute Z-scores for the spread
z_scores = (spread - mean_spread) / std_spread

# Set Z-score threshold for entry and exit
entry_threshold = 1.0
exit_threshold = 0.5

# Initialize position and portfolio value
position = 0  # 0 for no position, 1 for long position, -1 for short position

#portfolio_value = 0
covered_position_value = 0  # For tracking the value of covered positions

# Initialize variables to track profit and covered position profit
total_profit = 0
total_covered_position_profit = 0

# Loop through spread data
for i in range(1, len(spread)):
    z_score = z_scores[i]
    
    if position == 0:
        if z_score > entry_threshold:
            # Open short position
            position = -1
            covered_position_value = spread.iloc[i]  # Track the value of covered position
            print(f"Short XOM, Long CVX: Date={spread.index[i]}, Z-score={z_score:.2f}")
        elif z_score < -entry_threshold:
            # Open long position
            position = 1
            covered_position_value = -spread.iloc[i]  # Track the value of covered position
            print(f"Long XOM, Short CVX: Date={spread.index[i]}, Z-score={z_score:.2f}")
    elif position == 1:
        if z_score > -exit_threshold:
            # Close long position
            position = 0
            profit = spread.iloc[i] + covered_position_value
            total_profit += profit
            total_covered_position_profit += profit
            print(f"Exit Long XOM, Exit Short CVX: Date={spread.index[i]}, Z-score={z_score:.2f}, Profit={profit:.2f}")
    elif position == -1:
        if z_score < exit_threshold:
            # Close short position
            position = 0
            profit = covered_position_value - spread.iloc[i]
            total_profit += profit
            total_covered_position_profit += profit
            print(f"Exit Short XOM, Exit Long CVX: Date={spread.index[i]}, Z-score={z_score:.2f}, Profit={profit:.2f}")

# Print total profit

print(f"Total Profit: ${total_profit:.2f}")
print(f"Total Covered Position Profit: ${total_covered_position_profit:.2f}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Short XOM, Long CVX: Date=2019-01-03 00:00:00, Z-score=1.01
Exit Short XOM, Exit Long CVX: Date=2019-06-07 00:00:00, Z-score=0.46, Profit=6.53
Short XOM, Long CVX: Date=2020-03-09 00:00:00, Z-score=1.04
Exit Short XOM, Exit Long CVX: Date=2020-04-29 00:00:00, Z-score=0.42, Profit=7.34
Short XOM, Long CVX: Date=2020-09-23 00:00:00, Z-score=1.09
Exit Short XOM, Exit Long CVX: Date=2020-11-13 00:00:00, Z-score=0.45, Profit=7.60
Long XOM, Short CVX: Date=2022-02-25 00:00:00, Z-score=-1.11
Exit Long XOM, Exit Short CVX: Date=2022-07-12 00:00:00, Z-score=-0.47, Profit=7.50
Long XOM, Short CVX: Date=2022-07-29 00:00:00, Z-score=-1.48
Total Profit: $28.97
Total Covered Position Profit: $28.97


In [4]:
excel_file_name2 = 'xomdata.xlsx'
cvx_data.to_excel(excel_file_name2, index=True)
from IPython.display import FileLink
FileLink(excel_file_name2)

C:\Users\risha\xomdata.xlsx